# YOLO Fine-tuning on Mobile Phone Dataset
## Fine-tune YOLOv8 and Upload to Hugging Face

This notebook provides a complete pipeline for:
1. Downloading the mobile phone dataset from Kaggle
2. Converting Pascal VOC annotations to YOLO format
3. Fine-tuning YOLOv8 on the dataset
4. Pushing the model to Hugging Face Hub

## Step 1: Install Dependencies

In [42]:
# Install required packages
!pip install --upgrade pip
!pip install ultralytics opencv-python kagglehub huggingface-hub pillow torch torchvision pyyaml

# Verify installations
import ultralytics
print(f"YOLOv8 version: {ultralytics.__version__}")

YOLOv8 version: 8.4.11


## Step 2: Setup Kaggle Credentials

Choose one of two methods:
- **Method 1**: Paste your Kaggle API token directly
- **Method 2**: Upload kaggle.json file

In [43]:
import os
import getpass
from google.colab import files

print("\n" + "="*60)
print("KAGGLE AUTHENTICATION")
print("="*60)
print("\nChoose your authentication method:")
print("  1. Paste Kaggle API Token (RECOMMENDED)")
print("  2. Upload kaggle.json file")
print("\nYour token will NOT be saved or displayed.")

choice = input("\nEnter choice (1 or 2): ").strip()

if choice == "1":
    print("\n📝 Enter your Kaggle API token")
    print("   Get it from: https://www.kaggle.com/settings/account")
    print("   Click 'Create New API Token' and copy the token value\n")

    kaggle_token = ""

    if not kaggle_token:
        raise ValueError("Token cannot be empty!")

    os.environ['KAGGLE_API_TOKEN'] = kaggle_token

    os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

    print("\nKaggle credentials configured using API token!")
    print("   Token is stored in memory for this session only.")

elif choice == "2":
    print("\n📤 Please upload your kaggle.json file")
    print("   Get it from: https://www.kaggle.com/settings/account")
    print("   Click 'Create New API Token' to download kaggle.json\n")

    uploaded = files.upload()

    if 'kaggle.json' not in uploaded:
        raise FileNotFoundError("kaggle.json file not found!")

    # Setup Kaggle
    os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    print("\nKaggle credentials configured using kaggle.json!")
else:
    raise ValueError("Invalid choice. Please enter 1 or 2.")

print("\n" + "="*60)


KAGGLE AUTHENTICATION

Choose your authentication method:
  1. Paste Kaggle API Token (RECOMMENDED)
  2. Upload kaggle.json file

Your token will NOT be saved or displayed.

Enter choice (1 or 2): 1

📝 Enter your Kaggle API token
   Get it from: https://www.kaggle.com/settings/account
   Click 'Create New API Token' and copy the token value


Kaggle credentials configured using API token!
   Token is stored in memory for this session only.



## Step 3: Download Dataset from Kaggle

In [45]:
import kagglehub
import os

print("\n Downloading mobile phone dataset from Kaggle...")

try:
    dataset_path = kagglehub.dataset_download("dataclusterlabs/mobile-phone-image-dataset")
    print(f"\nDataset downloaded successfully!")
    print(f"Path: {dataset_path}")
except Exception as e:
    print(f"\nError downloading dataset: {str(e)}")
    print("\nTroubleshooting:")
    print("  1. Verify your Kaggle API token is correct")
    print("  2. Check that you have access to the dataset")
    print("  3. Ensure your internet connection is stable")
    raise

print("\nDataset contents:")
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        file_count = len(os.listdir(item_path))
        print(f"  {item}: {file_count} files")
    else:
        print(f"  {item}")


Using Colab cache for faster access to the 'mobile-phone-image-dataset' dataset.

Dataset downloaded successfully!
Path: /kaggle/input/mobile-phone-image-dataset

Dataset contents:
  Mobile_image: 1 files
  Annotations: 1 files


In [46]:
import os

# Check what's actually in the Annotations directory
annotations_dir = "/kaggle/input/mobile-phone-image-dataset/Annotations/Annotations"

print("=" * 70)
print("CHECKING ANNOTATION DIRECTORY")
print("=" * 70)

print(f"\nPath: {annotations_dir}")
print(f"Exists: {os.path.exists(annotations_dir)}")

if os.path.exists(annotations_dir):
    items = os.listdir(annotations_dir)
    print(f"Total items: {len(items)}\n")

    print("First 20 items:")
    for item in items[:20]:
        item_path = os.path.join(annotations_dir, item)
        if os.path.isdir(item_path):
            sub_count = len(os.listdir(item_path))
            print(f"  {item}/ ({sub_count} files)")
        else:
            _, ext = os.path.splitext(item)
            print(f"  {item} ({ext})")

    # Count file types
    xml_count = len([f for f in items if f.endswith('.xml')])
    json_count = len([f for f in items if f.endswith('.json')])

    print(f"\nSummary:")
    print(f"  XML files: {xml_count}")
    print(f"  JSON files: {json_count}")
    print(f"  Directories: {len([f for f in items if os.path.isdir(os.path.join(annotations_dir, f))])}")

print("\n" + "=" * 70)
print("FULL DIRECTORY TREE")
print("=" * 70)

for root, dirs, files in os.walk(annotations_dir):
    level = root.replace(annotations_dir, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent} {os.path.basename(root)}/")

    for f in files[:10]:
        print(f"{indent}    {f}")
    if len(files) > 10:
        print(f"{indent}  ... and {len(files) - 10} more")

    if level > 2:
        break

CHECKING ANNOTATION DIRECTORY

Path: /kaggle/input/mobile-phone-image-dataset/Annotations/Annotations
Exists: True
Total items: 100

First 20 items:
  Datacluster Labs Phone Dataset (49).xml (.xml)
  Datacluster Labs Phone Dataset (47).xml (.xml)
  Datacluster Labs Phone Dataset (65).xml (.xml)
  Datacluster Labs Phone Dataset (5).xml (.xml)
  Datacluster Labs Phone Dataset (94).xml (.xml)
  Datacluster Labs Phone Dataset (14).xml (.xml)
  Datacluster Labs Phone Dataset (4).xml (.xml)
  Datacluster Labs Phone Dataset (45).xml (.xml)
  Datacluster Labs Phone Dataset (90).xml (.xml)
  Datacluster Labs Phone Dataset (43).xml (.xml)
  Datacluster Labs Phone Dataset (30).xml (.xml)
  Datacluster Labs Phone Dataset (22).xml (.xml)
  Datacluster Labs Phone Dataset (15).xml (.xml)
  Datacluster Labs Phone Dataset (13).xml (.xml)
  Datacluster Labs Phone Dataset (19).xml (.xml)
  Datacluster Labs Phone Dataset (92).xml (.xml)
  Datacluster Labs Phone Dataset (23).xml (.xml)
  Datacluster Labs P

## Step 4: Convert Pascal VOC to YOLO Format

In [47]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path
import shutil
import random

# Configuration
DATASET_PATH = dataset_path
OUTPUT_DIR = "/content/yolo_dataset"
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

print("\n  Converting Pascal VOC to YOLO format...\n")

# Create output directories
os.makedirs(f"{OUTPUT_DIR}/images/train", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/images/val", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/images/test", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/labels/train", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/labels/val", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/labels/test", exist_ok=True)

# Class mapping
class_mapping = {'mobile_phone': 0}

def convert_voc_to_yolo(xml_file, image_width, image_height):
    """
    Convert VOC bounding box to YOLO format
    VOC format: xmin, ymin, xmax, ymax (pixel coordinates)
    YOLO format: x_center, y_center, width, height (normalized 0-1)
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_annotations = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in class_mapping:
            continue

        class_id = class_mapping[class_name]

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2.0 / image_width
        y_center = (ymin + ymax) / 2.0 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        # Clamp values to [0, 1]
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        width = max(0, min(1, width))
        height = max(0, min(1, height))

        yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_annotations

# Find annotation and image directories
annotations_dir = "/kaggle/input/mobile-phone-image-dataset/Annotations/Annotations"
images_source_dir = "/kaggle/input/mobile-phone-image-dataset/Mobile_image"

print(f"  Annotations directory: {annotations_dir}")
print(f"  Images directory: {images_source_dir}\n")

if not annotations_dir or not images_source_dir:
    print("   ERROR: Could not find annotations or images directory!")
    print(f"\nContents of {DATASET_PATH}:")
    print(os.listdir(DATASET_PATH))
    raise FileNotFoundError("Dataset directories not found")
else:
    print("  Scanning dataset...")

    # Collect all XML files
    xml_files = [f for f in os.listdir(annotations_dir) if f.endswith('.xml')]
    print(f"Found {len(xml_files)} annotation files\n")

    # Shuffle and split
    random.shuffle(xml_files)
    train_count = int(len(xml_files) * TRAIN_RATIO)
    val_count = int(len(xml_files) * VAL_RATIO)

    train_files = xml_files[:train_count]
    val_files = xml_files[train_count:train_count + val_count]
    test_files = xml_files[train_count + val_count:]

    print(f"  Dataset split:")
    print(f"  Train: {len(train_files)} samples")
    print(f"  Val:   {len(val_files)} samples")
    print(f"  Test:  {len(test_files)} samples\n")

    conversion_count = 0
    skipped_count = 0

    annotations_dir = None
    images_source_dir = None

    for item in os.listdir(DATASET_PATH):
      item_path = os.path.join(DATASET_PATH, item)
      if os.path.isdir(item_path):
          if 'annotation' in item.lower():
              # Check if there's a nested Annotations/Annotations structure
              potential_nested = os.path.join(item_path, item)
              if os.path.exists(potential_nested) and os.path.isdir(potential_nested):
                  annotations_dir = potential_nested
              else:
                  annotations_dir = item_path

          if 'image' in item.lower() or 'mobile' in item.lower():
              # Check if there's a nested Mobile_image/Mobile_image structure
              potential_nested = os.path.join(item_path, item)
              if os.path.exists(potential_nested) and os.path.isdir(potential_nested):
                  images_source_dir = potential_nested
              else:
                  images_source_dir = item_path


    conversion_count = 0
    skipped_count = 0
    error_count = 0

    for split, split_files in [("train", train_files), ("val", val_files), ("test", test_files)]:
        print(f"Processing {split} set...")
        for xml_file in split_files:
            try:
                xml_path = os.path.join(annotations_dir, xml_file)

                # Get image filename from XML and image dimensions
                tree = ET.parse(xml_path)
                root = tree.getroot()
                image_filename = root.find('filename').text

                size_elem = root.find('size')
                image_width = int(size_elem.find('width').text)
                image_height = int(size_elem.find('height').text)

                # Convert annotations
                yolo_annotations = convert_voc_to_yolo(xml_path, image_width, image_height)

                if not yolo_annotations:
                    skipped_count += 1
                    continue

                # Find and copy image
                image_source = os.path.join(images_source_dir, image_filename)
                if os.path.exists(image_source):
                    image_dest = os.path.join(OUTPUT_DIR, "images", split, image_filename)
                    shutil.copy2(image_source, image_dest)
                    # Save YOLO format annotations
                    label_filename = os.path.splitext(image_filename)[0] + ".txt"
                    label_dest = os.path.join(OUTPUT_DIR, "labels", split, label_filename)
                    with open(label_dest, 'w') as f:
                        f.write("\n".join(yolo_annotations))

                    conversion_count += 1
                else:
                    error_count += 1
            except Exception as e:
                skipped_count += 1

        print(f"  {split} set processed\n")




    print(f"\nConversion complete!")
    print(f"  Successfully converted: {conversion_count}")
    print(f"  Skipped: {skipped_count}")
    print(f"  Dataset: {OUTPUT_DIR}")


  Converting Pascal VOC to YOLO format...

  Annotations directory: /kaggle/input/mobile-phone-image-dataset/Annotations/Annotations
  Images directory: /kaggle/input/mobile-phone-image-dataset/Mobile_image

  Scanning dataset...
Found 100 annotation files

  Dataset split:
  Train: 80 samples
  Val:   10 samples
  Test:  10 samples

Processing train set...
  train set processed

Processing val set...
  val set processed

Processing test set...
  test set processed


Conversion complete!
  Successfully converted: 100
  Skipped: 0
  Dataset: /content/yolo_dataset


In [48]:
import os
from pathlib import Path

# Check what's in the downloaded dataset
print("=" * 70)
print("🔍 DATASET DIAGNOSTIC")
print("=" * 70)

print(f"\n  Main dataset path: {dataset_path}\n")

# List everything in the dataset
print("Contents of dataset directory:")
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        file_count = len(os.listdir(item_path))
        print(f"\n    {item}/ ({file_count} files)")

        for sub_item in os.listdir(item_path)[:5]:
            sub_path = os.path.join(item_path, sub_item)
            if os.path.isdir(sub_path):
                print(f"       {sub_item}/")
            else:
                print(f"       {sub_item}")
        if file_count > 5:
            print(f"     ... and {file_count - 5} more")
    else:
        print(f"\n    {item}")

# Show full directory tree
print("\n" + "=" * 70)
print("Full directory structure:")
print("=" * 70)
!find {dataset_path} -type d | head -30

🔍 DATASET DIAGNOSTIC

  Main dataset path: /kaggle/input/mobile-phone-image-dataset

Contents of dataset directory:

    Mobile_image/ (1 files)
       Mobile_image/

    Annotations/ (1 files)
       Annotations/

Full directory structure:
/kaggle/input/mobile-phone-image-dataset
/kaggle/input/mobile-phone-image-dataset/Mobile_image
/kaggle/input/mobile-phone-image-dataset/Mobile_image/Mobile_image
/kaggle/input/mobile-phone-image-dataset/Annotations
/kaggle/input/mobile-phone-image-dataset/Annotations/Annotations


## Step 5: Verify Dataset Structure

In [49]:
import os

print("\n  Dataset Structure Verification\n")

# Verify dataset structure
for split in ['train', 'val', 'test']:
    images_count = len(os.listdir(f"{OUTPUT_DIR}/images/{split}"))
    labels_count = len(os.listdir(f"{OUTPUT_DIR}/labels/{split}"))
    match = "success" if images_count == labels_count else "failure"
    print(f"{match} {split.upper():5} | Images: {images_count:4d} | Labels: {labels_count:4d}")

# Show sample annotation
print("\nSample annotation file:")
sample_label = f"{OUTPUT_DIR}/labels/train/{os.listdir(f'{OUTPUT_DIR}/labels/train')[0]}"
with open(sample_label, 'r') as f:
    content = f.read()
    print(f"  File: {os.path.basename(sample_label)}")
    print(f"  Content:\n{content}")


  Dataset Structure Verification

success TRAIN | Images:   97 | Labels:   97
success VAL   | Images:   19 | Labels:   19
success TEST  | Images:   18 | Labels:   18

Sample annotation file:
  File: Datacluster Labs Phone Dataset (32).txt
  Content:
0 0.4474038461538462 0.37786057692307695 0.5378846153846154 0.7557211538461539


## Step 6: Create YOLO Dataset Configuration File

In [50]:
import yaml
import os

# Dataset.yaml for YOLO
dataset_yaml = {
    'path': OUTPUT_DIR,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 1,
    'names': ['mobile_phone']
}

yaml_path = os.path.join(OUTPUT_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f)

print(f"\nDataset configuration created")
print(f"Path: {yaml_path}\n")
print("Configuration:")
print(yaml.dump(dataset_yaml))


Dataset configuration created
Path: /content/yolo_dataset/data.yaml

Configuration:
names:
- mobile_phone
nc: 1
path: /content/yolo_dataset
test: images/test
train: images/train
val: images/val



## Step 7: Fine-tune YOLOv8 Model

In [51]:
from ultralytics import YOLO
import os

print("\n" + "="*60)
print("STARTING MODEL TRAINING")
print("="*60 + "\n")

# Load pretrained YOLOv8 model
print("Loading pretrained YOLOv8n model...\n")
model = YOLO('yolov8n.pt')  # nano model - fastest, good for mobile

# Training configuration
epochs = 50  # Adjust based on your needs
imgsz = 640
batch_size = 16  # Adjust based on GPU memory
patience = 10  # Early stopping patience

print(f"    Training configuration:")
print(f"  • Model: YOLOv8n (Nano)")
print(f"  • Epochs: {epochs}")
print(f"  • Image size: {imgsz}x{imgsz}")
print(f"  • Batch size: {batch_size}")
print(f"  • Early stopping patience: {patience} epochs")
print(f"  • Dataset: {yaml_path}\n")

print("   Starting training... This may take 2-3 hours on T4 GPU\n")

# Train the model
results = model.train(
    data=yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    patience=patience,
    device=0,  # GPU device
    project='mobile_phone_detector',
    name='yolov8n_v1',
    exist_ok=False,
    verbose=True,
    save=True,
    save_period=5,
    plots=True,
    conf=0.5,
    iou=0.45
)

print("\n" + "="*60)
print("   TRAINING COMPLETED!")
print("="*60)


STARTING MODEL TRAINING

Loading pretrained YOLOv8n model...

    Training configuration:
  • Model: YOLOv8n (Nano)
  • Epochs: 50
  • Image size: 640x640
  • Batch size: 16
  • Early stopping patience: 10 epochs
  • Dataset: /content/yolo_dataset/data.yaml

   Starting training... This may take 2-3 hours on T4 GPU

Ultralytics 8.4.11 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.5, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64

## Step 8: Evaluate Model

In [54]:
import os
from ultralytics import YOLO

# List what's in the working directory
print("Contents of /content:")
print(os.listdir('/content'))

# Check if mobile_phone_detector exists
if os.path.exists('/content/mobile_phone_detector'):
    print("\nContents of mobile_phone_detector:")
    for root, dirs, files in os.walk('/content/mobile_phone_detector'):
        level = root.replace('/content/mobile_phone_detector', '').count(os.sep)
        indent = '  ' * level
        print(f"{indent}{os.path.basename(root)}/")
        for file in files[:5]:
            print(f"{indent}  {file}")

Contents of /content:
['.config', 'yolov8n.pt', 'yolo26n.pt', 'yolo_dataset', 'runs', 'sample_data']


In [55]:
import os

print("Contents of /content/runs:")
for root, dirs, files in os.walk('/content/runs'):
    level = root.replace('/content/runs', '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/")
    for file in files[:10]:
        print(f"{indent}  {file}")
    if level > 2:  # Limit depth
        break

Contents of /content/runs:
runs/
  detect/
    mobile_phone_detector/
      yolov8n_v12/
        args.yaml
        BoxF1_curve.png
        BoxR_curve.png
        train_batch282.jpg
        train_batch2.jpg
        val_batch0_labels.jpg
        train_batch280.jpg
        train_batch1.jpg
        confusion_matrix_normalized.png
        confusion_matrix.png


In [56]:
from ultralytics import YOLO

print("\n  EVALUATING MODEL\n")

# Load the best model
best_model = YOLO('/content/runs/detect/mobile_phone_detector/yolov8n_v1/weights/best.pt')

# Validate on test set
print("Running validation on test set...\n")
metrics = best_model.val()

print("\n" + "="*60)
print("   VALIDATION RESULTS")
print("="*60)
print(f"  mAP50 (0.50 IOU):   {metrics.box.map50:.3f}")
print(f"  mAP50-95:           {metrics.box.map:.3f}")
print(f"  Precision:          {metrics.box.mp:.3f}")
print(f"  Recall:             {metrics.box.mr:.3f}")
print("="*60)


  EVALUATING MODEL

Running validation on test set...

Ultralytics 8.4.11 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2971.5±1716.7 MB/s, size: 4801.4 KB)
val: Scanning /content/yolo_dataset/labels/val.cache... 19 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 19/19 8.9Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 1.7s/it 3.5s
                   all         19         19      0.946          1      0.993      0.903
Speed: 8.9ms preprocess, 20.6ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /content/runs/detect/val2

   VALIDATION RESULTS
  mAP50 (0.50 IOU):   0.993
  mAP50-95:           0.903
  Precision:          0.946
  Recall:             1.000


## Step 9: Setup Hugging Face Credentials

In [57]:
from huggingface_hub import login
import getpass

print("\n" + "="*60)
print("🤗 HUGGING FACE AUTHENTICATION")

hf_token =""

# Login to Hugging Face
try:
    login(token=hf_token)
    print("\n  Logged in to Hugging Face successfully!")
except Exception as e:
    print(f"\n  Authentication failed: {str(e)}")
    raise


🤗 HUGGING FACE AUTHENTICATION

  Logged in to Hugging Face successfully!


## Step 10: Push Model to Hugging Face

In [58]:
from huggingface_hub import HfApi, ModelCard, ModelCardData
from pathlib import Path
import os
import json
from datetime import datetime

# Configuration
MODEL_NAME = "yolov8n-mobile-phone"
REPO_ID = f"IndUSV/{MODEL_NAME}"

print("\n" + "="*60)
print("  PUSHING MODEL TO HUGGING FACE")
print("="*60)
print(f"\n  Target repository: {REPO_ID}\n")

# Create the repo
api = HfApi()

try:
    api.create_repo(repo_id=REPO_ID, repo_type="model", exist_ok=True, private=False)
    print(f"  Repository {REPO_ID} is ready")
except Exception as e:
    print(f"  Repository creation note: {str(e)}")

# Prepare model files
model_path = Path('/content/runs/detect/mobile_phone_detector/yolov8n_v1')
best_model_path = model_path / 'weights' / 'best.pt'

if not best_model_path.exists():
    print(f"\n  Error: Model file not found at {best_model_path}")
else:
    print(f"\n  Model file found: {best_model_path}")
    print(f"   Size: {best_model_path.stat().st_size / (1024*1024):.1f} MB\n")

    # Upload the model
    print("  Uploading model weights...")
    try:
        api.upload_file(
            path_or_fileobj=str(best_model_path),
            path_in_repo="pytorch_model.bin",
            repo_id=REPO_ID,
            repo_type="model"
        )
        print("   Model weights uploaded!\n")
    except Exception as e:
        print(f"  Upload error: {str(e)}\n")

    # Upload training results and metrics
    results_file = model_path / 'results.csv'
    if results_file.exists():
        print("  Uploading training results...")
        try:
            api.upload_file(
                path_or_fileobj=str(results_file),
                path_in_repo="results.csv",
                repo_id=REPO_ID,
                repo_type="model"
            )
            print("  Training results uploaded!\n")
        except Exception as e:
            print(f"  Results upload note: {str(e)}\n")


  PUSHING MODEL TO HUGGING FACE

  Target repository: IndUSV/yolov8n-mobile-phone

  Repository IndUSV/yolov8n-mobile-phone is ready

  Model file found: /content/runs/detect/mobile_phone_detector/yolov8n_v1/weights/best.pt
   Size: 6.0 MB

  Uploading model weights...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...olov8n_v1/weights/best.pt: 100%|##########| 6.25MB / 6.25MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


   Model weights uploaded!

  Uploading training results...
  Training results uploaded!



## Step 11: Create Model Card for Hugging Face

In [38]:
from huggingface_hub import HfApi
from datetime import datetime

# Create a comprehensive model card
model_card_content = f"""
---
license: mit
tags:
- yolov8
- object-detection
- mobile-phone
- computer-vision
---

# YOLOv8n Mobile Phone Detector

A fine-tuned YOLOv8 Nano model trained on the Datacluster Labs Mobile Phone Image Dataset for object detection of mobile phones in images.

## Model Details

- **Base Model**: YOLOv8n (Nano)
- **Task**: Object Detection
- **Dataset**: Datacluster Labs Mobile Phone Image Dataset
- **Classes**: mobile_phone (1 class)
- **Training Date**: {datetime.now().strftime('%Y-%m-%d')}

## Usage

```python
from ultralytics import YOLO

# Load model from Hugging Face
model = YOLO('huggingface://IndUSV/yolov8n-mobile-phone-detector/pytorch_model.bin')

# Run inference
results = model.predict(source='image.jpg', conf=0.5)
```

## Training Details

- **Input Size**: 640x640
- **Batch Size**: 16
- **Optimizer**: SGD
- **Learning Rate**: Auto
- **Epochs**: 50 (with early stopping)
- **Device**: NVIDIA GPU

## Dataset Information

The model was trained on the Datacluster Labs Mobile Phone Image Dataset which contains:
- High-resolution mobile phone images
- Pascal VOC format annotations
- Diverse backgrounds and lighting conditions
- Various phone models and orientations

Dataset splits:
- Training: 80%
- Validation: 10%
- Testing: 10%

## Performance

Check results.csv for detailed training metrics and evaluation results.

## Citation

If you use this model, please cite:
- YOLOv8: https://github.com/ultralytics/ultralytics
- Dataset: https://www.kaggle.com/datasets/dataclusterlabs/mobile-phone-image-dataset

## License

This model is released under the MIT License.
"""

# Upload model card
api = HfApi()

print("\n  Creating and uploading model card...")
try:
    api.upload_file(
        path_or_fileobj=model_card_content.encode('utf-8'),
        path_in_repo="README.md",
        repo_id=REPO_ID,
        repo_type="model"
    )
    print(f"  Model card uploaded!\n")
except Exception as e:
    print(f". Model card note: {str(e)}\n")

print("="*60)
print("  MODEL SUCCESSFULLY PUSHED TO HUGGING FACE")
print("="*60)
print(f"\n  Model URL: https://huggingface.co/{REPO_ID}")
print(f"  Access your models: https://huggingface.co/IndUSV/models")


📝 Creating and uploading model card...
✅ Model card uploaded!

✅ MODEL SUCCESSFULLY PUSHED TO HUGGING FACE

🌐 Model URL: https://huggingface.co/IndUSV/yolov8n-mobile-phone
📝 Access your models: https://huggingface.co/IndUSV/models


## Step 12: Test Inference with Fine-tuned Model

In [40]:
from ultralytics import YOLO
import os

print("\n" + "="*60)
print("🧪 TESTING INFERENCE")
print("="*60 + "\n")

# Load the best model
model = YOLO('/content/runs/detect/mobile_phone_detector/yolov8n_v1/weights/best.pt')

# Run inference on test images
test_images = os.listdir(f"{OUTPUT_DIR}/images/test")[:3]  # Get first 3 test images

print(f"Running inference on {len(test_images)} test images...\n")

for idx, img_file in enumerate(test_images, 1):
    img_path = os.path.join(OUTPUT_DIR, 'images', 'test', img_file)

    # Run inference
    results = model.predict(source=img_path, conf=0.5, verbose=False)

    # Display results
    result = results[0]

    print(f"Test {idx}: {img_file}")
    print(f"    Detections: {len(result.boxes)}")

    if len(result.boxes) > 0:
        for box_idx, box in enumerate(result.boxes, 1):
            conf = box.conf.item()
            print(f"    • Detection {box_idx}: Confidence {conf:.3f}")
    print()

print("="*60)
print("  Inference test completed!")
print("="*60)


🧪 TESTING INFERENCE

Running inference on 3 test images...

Test 1: Datacluster Labs Phone Dataset (46).jpg
  📱 Detections: 1
    • Detection 1: Confidence 0.976

Test 2: Datacluster Labs Phone Dataset (22).jpg
  📱 Detections: 1
    • Detection 1: Confidence 0.933

Test 3: Datacluster Labs Phone Dataset (9).jpg
  📱 Detections: 1
    • Detection 1: Confidence 0.906

✅ Inference test completed!


## Step 13: Final Summary

In [41]:
print("\n")
print("╔" + "="*58 + "╗")
print("║" + " "*10 + "YOLO FINE-TUNING PIPELINE COMPLETED!" + " "*12 + "║")
print("╚" + "="*58 + "╝")

print("\n  COMPLETED STEPS:")
print("  1. ✓ Downloaded mobile phone dataset from Kaggle")
print("  2. ✓ Converted Pascal VOC annotations to YOLO format")
print("  3. ✓ Fine-tuned YOLOv8n model")
print("  4. ✓ Evaluated model performance")
print("  5. ✓ Pushed model to Hugging Face Hub")
print("  6. ✓ Created comprehensive model card")
print("  7. ✓ Tested inference on sample images")

print(f"\n  MODEL INFORMATION:")
print(f"  Model URL:    https://huggingface.co/{REPO_ID}")
print(f"  Model ID:     {REPO_ID}")
print(f"  Base Model:   YOLOv8n (Nano)")
print(f"  Classes:      1 (mobile_phone)")
print(f"  Framework:    Ultralytics YOLOv8")

print(f"\n  LOCAL ARTIFACTS:")
print(f"  Training logs:  ./mobile_phone_detector/yolov8n_v1/")
print(f"  Best weights:   ./mobile_phone_detector/yolov8n_v1/weights/best.pt")
print(f"  Dataset:        {OUTPUT_DIR}")

print(f"\n  NEXT STEPS:")
print(f"  1. Download model: model = YOLO('IndUSV/{MODEL_NAME}')")
print(f"  2. Run inference: results = model.predict('image.jpg')")
print(f"  3. Export model: model.export(format='onnx')")
print(f"  4. Deploy on edge devices")

print(f"\n  DOCUMENTATION:")
print(f"  YOLOv8:        https://docs.ultralytics.com/")
print(f"  Dataset:       https://www.kaggle.com/datasets/dataclusterlabs/mobile-phone-image-dataset")
print(f"  Hugging Face:  https://huggingface.co/IndUSV/models")

print("\n" + "═"*60)



╔==========================================================╗
║          YOLO FINE-TUNING PIPELINE COMPLETED!            ║
╚==========================================================╝

✅ COMPLETED STEPS:
  1. ✓ Downloaded mobile phone dataset from Kaggle
  2. ✓ Converted Pascal VOC annotations to YOLO format
  3. ✓ Fine-tuned YOLOv8n model
  4. ✓ Evaluated model performance
  5. ✓ Pushed model to Hugging Face Hub
  6. ✓ Created comprehensive model card
  7. ✓ Tested inference on sample images

📊 MODEL INFORMATION:
  Model URL:    https://huggingface.co/IndUSV/yolov8n-mobile-phone
  Model ID:     IndUSV/yolov8n-mobile-phone
  Base Model:   YOLOv8n (Nano)
  Classes:      1 (mobile_phone)
  Framework:    Ultralytics YOLOv8

📁 LOCAL ARTIFACTS:
  Training logs:  ./mobile_phone_detector/yolov8n_v1/
  Best weights:   ./mobile_phone_detector/yolov8n_v1/weights/best.pt
  Dataset:        /content/yolo_dataset

🚀 NEXT STEPS:
  1. Download model: model = YOLO('IndUSV/yolov8n-mobile-phone')
  2. R